In [ ]:
# @title
!pip install transformer-lens

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00


In [ ]:
# Import stuff
import torch
import os
import unicodedata
import json
import string
import re
from transformers import AutoTokenizer, GPTNeoXForCausalLM
from transformer_lens import HookedTransformer

torch.set_grad_enabled(False)

def make_prompt(prompt_template: str, subject: str) -> str:
    prompt = prompt_template.format(subject)
    return prompt

def normalize(s):
    """Normalize answer."""
    s = unicodedata.normalize("NFD", s)

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    s = white_space_fix(remove_articles(remove_punc(lower(s))))
    return s
def compute_score(prediction, truth):
    pred_tokens = normalize(prediction).split()
    truth_tokens = normalize(truth).split()

    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)
    if len(common_tokens) == 0:
        return 0
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    return rec

# Load counterfact dataset7
data_pth = './counterfact_data.json'
with open(data_pth, 'r') as f:
    counterfact_data = json.load(f)

# Load model from Hugging Face (smaller pythia model)
MODEL_NAME = "EleutherAI/pythia-70m"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
hf_model = GPTNeoXForCausalLM.from_pretrained(
    MODEL_NAME,
    low_cpu_mem_usage=True,
    device_map="auto"  # allow automatic layer loading
)

# Load with transformer_lens
print("Loading HookedTransformer...")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = HookedTransformer.from_pretrained(
    "EleutherAI/pythia-70m",
    hf_model=hf_model,
    device=device,
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    tokenizer=tokenizer
)

# Create output directory if not exists
os.makedirs('./model_output', exist_ok=True)

# Run evaluation loop
for case in counterfact_data:
    recall_score = []
    outputs = []
    for p in case['prompts']:
        output = model.generate(p, max_new_tokens=10, temperature=0, prepend_bos=True)
        output = output.replace(p, '')
        recall_score.append(compute_score(output, case['target']))
        outputs.append(output)

    case.update({'recall': recall_score, 'output': outputs})

    with open('./model_output/pythia-result.jsonl', 'a+') as f:
        f.write(json.dumps(case) + '\n')

    # Check for hallucination (variation in recall score)
    if len(set(recall_score)) > 1:
        with open('./model_output/pythia-result-hall.jsonl', 'a+') as f:
            f.write(json.dumps(case) + '\n')

Loading HookedTransformer...
Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
!pip install git+https://github.com/AlignmentResearch/tuned-lens.git

  Cloning https://github.com/AlignmentResearch/tuned-lens.git to /tmp/pip-req-build-oh1wj65o
  Running command git clone --filter=blob:none --quiet https://github.com/AlignmentResearch/tuned-lens.git /tmp/pip-req-build-oh1wj65o
  Resolved https://github.com/AlignmentResearch/tuned-lens.git to commit 550644a07ed193007d006797b1ffe15d0a5c0cd2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
import torch
from tqdm import tqdm
import numpy as np
import pickle, json
from transformers import AutoTokenizer, GPTNeoXForCausalLM
from transformer_lens import HookedTransformer
from tuned_lens import TunedLens
MODEL_PATH = "EleutherAI/pythia-70m"  # Using Hugging Face model

import random
from datetime import datetime

# Load tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
hf_model = GPTNeoXForCausalLM.from_pretrained(MODEL_PATH, low_cpu_mem_usage=True)

# Disable gradient calculations for inference
torch.set_grad_enabled(False)

# Set device (CUDA if available, otherwise CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Path for the tuned lens (local path to your checkpoint)
tuned_lens_pth = 'EleutherAI/pythia-70m-deduped'

# Set up the environment variable for Hugging Face mirror
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# Load the model using HookedTransformer
model = HookedTransformer.from_pretrained(
    "EleutherAI/pythia-70m",  # Model name
    hf_model=hf_model,  # Pretrained model from Hugging Face
    device=device,  # Device (CPU or GPU)
    n_devices=1,  # Use 1 device
    move_to_device=True,  # Move the model to the device
    fold_ln=False,  # Option to fold layer norm weights
    center_writing_weights=False,  # Don't center the writing weights
    center_unembed=False,  # Don't center unembed
    tokenizer=tokenizer  # Use the tokenizer
)

# Load the tuned lens from the local file
lens = TunedLens.from_model_and_pretrained(model, tuned_lens_pth)
lens = lens.to(device)

# Import your utility functions
from get_dist import *
from plot_utils import *
from data_utils import *

def get_logits_dist(prompt, output, model, target, device, incl_mid=True, tok_w_space=False):
    if target.startswith(' ') and not tok_w_space:
        first_tok_idx = 2
    elif target[0].isdigit():  # if it starts with a digit
        first_tok_idx = 2
    else:
        first_tok_idx = 1

    llama_tokens = model.to_tokens(prompt)
    ans_tokens = model.to_tokens(target)
    llama_logits, llama_cache = model.run_with_cache(llama_tokens, remove_batch_dim=True)
    llama_tokens_str = model.to_str_tokens(prompt)
    answer_toks_wo_head = torch.tensor([ans_tokens[0][first_tok_idx]]).to(device)
    ans_str = model.to_str_tokens(target)[first_tok_idx]

    accumulated_residual, acc_labels = llama_cache.accumulated_resid(layer=-1, incl_mid=incl_mid, return_labels=True, apply_ln=True)
    scaled_residual_stack = llama_cache.apply_ln_to_stack(accumulated_residual, layer=-1, pos_slice=-1)

    unembed_res = model.unembed(model.ln_final(scaled_residual_stack))
    dist = torch.softmax(unembed_res, dim=-1)
    prob_ans = dist[:, :, answer_toks_wo_head[0]]

    return prob_ans, llama_tokens_str, ans_str

def get_tuned_logits_dist(lens, prompt, model, target, device, tok_w_space=False):
    if target.startswith(' ') and not tok_w_space:
        first_tok_idx = 2
    elif target[0].isdigit():  # if it starts with a digit
        first_tok_idx = 2
    else:
        first_tok_idx = 1

    llama_tokens = model.to_tokens(prompt)
    ans_tokens = model.to_tokens(target)
    logits, cache = model.run_with_cache(llama_tokens)
    llama_tokens_str = model.to_str_tokens(prompt)
    answer_toks_wo_head = torch.tensor([ans_tokens[0][first_tok_idx]]).to(device)
    ans_str = model.to_str_tokens(target)[first_tok_idx]

    predictition_traj_cache = PredictionTrajectory.from_lens_and_cache(
        lens=lens,
        cache=cache,
        model_logits=logits,
        input_ids=llama_tokens,
    )
    prob_ans = torch.exp(torch.tensor(predictition_traj_cache.log_probs[0][:, :, answer_toks_wo_head[0]]))

    return prob_ans, llama_tokens_str, ans_str

def get_residual_prob(data, model, target='object'):
    max_tok_len = 0
    tuned_ans_list = []
    logit_ans_list = []
    for d in tqdm(data):
        if target == 'object':
            d_target = ' ' + d[target]
        elif target == 'output':
            d_target = d[target]

        tuned_prob_ans, llama_tokens_str, ans_str = get_tuned_logits_dist(lens, d['prompt'], model, d_target, device, tok_w_space=True)
        logit_prob_ans, llama_tokens_str, ans_str = get_logits_dist(d['prompt'], d['output'], model, d_target, device, incl_mid=False, tok_w_space=True)

        max_tok_len = max(max_tok_len, len(llama_tokens_str))
        tuned_ans_list.append(tuned_prob_ans.cpu().numpy())
        logit_ans_list.append(logit_prob_ans.cpu().numpy())

    return tuned_ans_list, logit_ans_list, max_tok_len

def main():
    data_all = []
    data_r = []
    data_z = []
    data_pth_list = ['/content/pythia-result-hall.jsonl']

    for pth in data_pth_list:
        with open(pth, 'r') as f:
            for line in f:
                line_d = json.loads(line)
                data_all.append(line_d)

    from operator import itemgetter
    from itertools import groupby
    grouped_data = {}
    data_all.sort(key=itemgetter('relation_id'))
    for key, group in groupby(data_all, key=itemgetter('relation_id')):
        grouped_data[str(key)] = list(group)

    file_pth = './results/pythia-70m/tuned_logits/'
    os.makedirs(file_pth, exist_ok=True)

    for relation_id, relation_data in grouped_data.items():
        print('processing...  ', relation_id)
        data_r, data_z = get_right_zero_from_relation(relation_data)
        sample_size = min(len(data_r), len(data_z))
        sample_r = random.sample(data_r, sample_size)
        sample_z = random.sample(data_z, sample_size)

        pkl_pth = file_pth + relation_id + '.pkl'

        if os.path.exists(pkl_pth):
            print('loading from pkl...')
            with open(pkl_pth, 'rb') as f:
                tuned_ans_list_r, logit_ans_list_r, max_tok_len_r, tuned_ans_list_z, logit_ans_list_z, max_tok_len_z, tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o, sample_r, sample_z = pickle.load(f)
        else:
            print('processing...')
            tuned_ans_list_r, logit_ans_list_r, max_tok_len_r = get_residual_prob(sample_r, model)
            tuned_ans_list_z, logit_ans_list_z, max_tok_len_z = get_residual_prob(sample_z, model)
            tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o = get_residual_prob(sample_z, model, 'output')

            with open(pkl_pth, 'wb') as f:
                pickle.dump([tuned_ans_list_r, logit_ans_list_r, max_tok_len_r, tuned_ans_list_z, logit_ans_list_z, max_tok_len_z, tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o, sample_r, sample_z], f)

if __name__ == "__main__":
    main()

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
processing...   P101


100%|██████████| 8/8 [00:00<00:00, 19000.24it/s]


processing...


100%|██████████| 10/10 [00:00<00:00, 12.95it/s]


processing...   P103


...

100%|██████████| 103/103 [00:08<00:00, 11.81it/s]


processing...   P937


100%|██████████| 52/52 [00:00<00:00, 64413.41it/s]


processing...


100%|██████████| 63/63 [00:05<00:00, 10.73it/s]


In [ ]:
import os
import torch
from tqdm import tqdm
import numpy as np
import pickle, json
from tuned_lens import TunedLens

In [ ]:
import sys
from plot_utils import *
from get_dist import *

# both tuned and logit lens

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

file_pth = './results/pythia-70m/tuned_logits/'


l_data = []
label = []
for pkl_pth in os.listdir(file_pth):
    if pkl_pth[-4:] == '.pkl':
        with open(file_pth+pkl_pth, 'rb') as f:
            tuned_ans_list_r, logit_ans_list_r, max_tok_len_r, tuned_ans_list_z, logit_ans_list_z, max_tok_len_z, tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o, sample_r, sample_z = pickle.load(f)
        for i in range(len(tuned_ans_list_r)):
            l_data.append(np.concatenate((tuned_ans_list_r[i][:,-1],logit_ans_list_r[i][:,-1])))
            # l_data.append(logit_ans_list_r[i][:,-1])
            label.append(1)
        for i in range(len(tuned_ans_list_z_o)):
            l_data.append(np.concatenate((tuned_ans_list_z_o[i][:,-1],logit_ans_list_z_o[i][:,-1])))
            # l_data.append(logit_ans_list_z_o[i][:,-1])
            label.append(0)
print(len(l_data))
X_train, X_test, y_train, y_test = train_test_split(l_data, label, test_size=0.2, random_state=42)

clf = svm.SVC(kernel='linear')

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)

6404
accuracy: 0.8446526151444185


# logit lens

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

file_pth = './results/pythia-70m/tuned_logits/'


l_data = []
label = []
for pkl_pth in os.listdir(file_pth):
    if pkl_pth[-4:] == '.pkl':
        with open(file_pth+pkl_pth, 'rb') as f:
            tuned_ans_list_r, logit_ans_list_r, max_tok_len_r, tuned_ans_list_z, logit_ans_list_z, max_tok_len_z, tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o, sample_r, sample_z = pickle.load(f)
        for i in range(len(tuned_ans_list_r)):
            #l_data.append(np.concatenate((tuned_ans_list_r[i][:,-1],logit_ans_list_r[i][:,-1])))
            l_data.append(logit_ans_list_r[i][:,-1])
            label.append(1)
        for i in range(len(tuned_ans_list_z_o)):
            #l_data.append(np.concatenate((tuned_ans_list_z_o[i][:,-1],logit_ans_list_z_o[i][:,-1])))
            l_data.append(logit_ans_list_z_o[i][:,-1])
            label.append(0)
print(len(l_data))
X_train, X_test, y_train, y_test = train_test_split(l_data, label, test_size=0.2, random_state=42)

clf = svm.SVC(kernel='linear')

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)

6404
accuracy: 0.8235753317720531


# tuned lens

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

file_pth = './results/pythia-70m/tuned_logits/'


l_data = []
label = []
for pkl_pth in os.listdir(file_pth):
    if pkl_pth[-4:] == '.pkl':
        with open(file_pth+pkl_pth, 'rb') as f:
            tuned_ans_list_r, logit_ans_list_r, max_tok_len_r, tuned_ans_list_z, logit_ans_list_z, max_tok_len_z, tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o, sample_r, sample_z = pickle.load(f)
        for i in range(len(tuned_ans_list_r)):
            #l_data.append(np.concatenate((tuned_ans_list_r[i][:,-1],logit_ans_list_r[i][:,-1])))
            l_data.append(tuned_ans_list_r[i][:,-1])
            label.append(1)
        for i in range(len(tuned_ans_list_z_o)):
            #l_data.append(np.concatenate((tuned_ans_list_z_o[i][:,-1],logit_ans_list_z_o[i][:,-1])))
            l_data.append(tuned_ans_list_z_o[i][:,-1])
            label.append(0)
print(len(l_data))
X_train, X_test, y_train, y_test = train_test_split(l_data, label, test_size=0.2, random_state=42)

clf = svm.SVC(kernel='linear')

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)

6404
accuracy: 0.8017174082747853
